<a href="https://colab.research.google.com/github/IsraelAbebe/Personal-Projects-and-Exercises/blob/master/General-NLP/Kaggle_Semantic_Similarity_with_TF_Hub_Universal_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [0]:
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from textblob import TextBlob


import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam,RMSprop,Adagrad
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

def get_sentiment(message):
    result = TextBlob(message).sentiment.subjectivity
    return result
def get_sentiment2(message):
    result = TextBlob(message).sentiment.polarity
    return result+1
# Reduce logging output.
def get_sentese_embedding(messages):
    logging.set_verbosity(logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      return message_embeddings

In [0]:
paragraph = ["Universal Sentence Encoder embeddings also support short paragraphs. ",
    "There is no hard limit on how long the paragraph is. Roughly, the longer ",
    "the more 'diluted' the embedding will be."]

In [0]:
np.vectorize(get_sentiment2)(paragraph).reshape(-1,1)

array([[1.        ],
       [0.99861111],
       [1.5       ]])

In [0]:
get_sentese_embedding(paragraph).shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(3, 512)

In [0]:
def process_text(text):
    text = text.encode('ascii', errors='ignore').decode()
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'#+', ' ', text )
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
    text = re.sub(r"\'s", " ", text)     
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"isn't", "is not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

In [0]:
train = pd.read_csv("gdrive/My Drive/dataset/train.csv")
test = pd.read_csv("gdrive/My Drive/dataset/test.csv")
submission = pd.read_csv("gdrive/My Drive/dataset/sample_submission.csv")

In [0]:
train['text'] = train.text.apply(lambda x: process_text(x))
test['text'] = test.text.apply(lambda x: process_text(x))

train['count'] = train.text.apply(lambda x: len(x.split()))
test['count'] = test.text.apply(lambda x: len(x.split()))

In [0]:
sentiments_list = np.vectorize(get_sentiment)(train.text.values).reshape(-1,1)
sentiments_list2 = np.vectorize(get_sentiment2)(train.text.values).reshape(-1,1)
print(sentiments_list.shape)

test_sentiments_list = np.vectorize(get_sentiment)(test.text.values).reshape(-1,1)
test_sentiments_list2 = np.vectorize(get_sentiment2)(test.text.values).reshape(-1,1)
test_sentiments_list.shape

(7613, 1)


(3263, 1)

In [0]:
embeddings = get_sentese_embedding(train.text.values)
print(embeddings.shape)

test_embeddings = get_sentese_embedding(test.text.values)
print(test_embeddings.shape)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(7613, 512)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(3263, 512)


In [0]:
embeddings[:,:100].shape

(7613, 100)

In [0]:
final = pd.concat([pd.DataFrame(embeddings),pd.DataFrame(sentiments_list),pd.DataFrame(sentiments_list2),pd.DataFrame(train['count'].values)],axis=1)
print(final.values.shape)

test_final = pd.concat([pd.DataFrame(test_embeddings),pd.DataFrame(test_sentiments_list),pd.DataFrame(test_sentiments_list2),pd.DataFrame(test['count'].values)],axis=1)
test_final.values.shape

(7613, 515)


(3263, 515)

In [0]:
train_data = final.values
train_labels = train.target.values


test_data = test_final.values
train_data.shape,train_labels.shape,test_data.shape

((7613, 515), (7613,), (3263, 515))

In [0]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.svm import LinearSVC
# from sklearn.feature_selection import SelectFromModel

# lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_data, train_labels)
# model = SelectFromModel(lsvc, prefit=True)
# train_data = model.transform(train_data)

In [0]:
def build_model():
    model = Sequential([
        Input(shape=(515,)),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model = build_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 1024)              528384    
_________________________________________________________________
batch_normalization_12 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_13 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)              

In [0]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

train_history = model.fit(
    train_data, train_labels,
    validation_split=0.01,
    epochs=50,
    callbacks=[checkpoint,callback],
    batch_size=32
)

Train on 7536 samples, validate on 77 samples
Epoch 1/50
7536/7536 [==============================] - 1s 180us/sample - loss: 0.4409 - acc: 0.8166 - val_loss: 0.1825 - val_acc: 0.9221
Epoch 2/50
7536/7536 [==============================] - 1s 189us/sample - loss: 0.4405 - acc: 0.8121 - val_loss: 0.1783 - val_acc: 0.9351
Epoch 3/50
7536/7536 [==============================] - 1s 185us/sample - loss: 0.4376 - acc: 0.8141 - val_loss: 0.1732 - val_acc: 0.9351
Epoch 4/50
7536/7536 [==============================] - 1s 170us/sample - loss: 0.4247 - acc: 0.8183 - val_loss: 0.1780 - val_acc: 0.9481
Epoch 5/50
7536/7536 [==============================] - 1s 183us/sample - loss: 0.4196 - acc: 0.8259 - val_loss: 0.1778 - val_acc: 0.9221
Epoch 6/50
7536/7536 [==============================] - 1s 176us/sample - loss: 0.4187 - acc: 0.8174 - val_loss: 0.1636 - val_acc: 0.9481
Epoch 7/50
7536/7536 [==============================] - 1s 180us/sample - loss: 0.4081 - acc: 0.8280 - val_loss: 0.1692 - val_

In [0]:
model.load_weights('model.h5')
test_pred = model.predict(test_data)

In [0]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('gdrive/My Drive/dataset/submission.csv', index=False)